# Record environment

In [28]:
import sys
import os

print(sys.version)
with os.popen('git describe --tags --always --dirty=" (dirty)"') as out:
    print("Git commit:", "\n".join(out))

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# Repeatable random numbers
seed = 31337
print(f"Seeding pytorch with {seed}")
torch.manual_seed(seed)
# Avoid nondeterministic algorithms (may be slower)
torch.use_deterministic_algorithms(True)

3.9.4 (tags/v3.9.4:1f2e308, Apr  4 2021, 13:27:16) [MSC v.1928 64 bit (AMD64)]
Git commit: a0895b6 (dirty)

Seeding pytorch with 31337


TypeError: use_deterministic_algorithms() missing 1 required positional argument: 'mode'

# Fetch data

In [ ]:
# Download data from open datasets
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

# This stuff relies on the downloaded test data
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)



# Define model

In [ ]:
# Detect what local device we have available
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

Using cpu device


# Train the model

In [ ]:


# Training function
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

# Testing function
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

# Perform training
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")


Epoch 1
-------------------------------
loss: 2.294629  [   64/60000]
loss: 2.275685  [ 6464/60000]
loss: 2.258202  [12864/60000]
loss: 2.261698  [19264/60000]
loss: 2.241029  [25664/60000]
loss: 2.214132  [32064/60000]
loss: 2.212466  [38464/60000]
loss: 2.173881  [44864/60000]
loss: 2.170368  [51264/60000]
loss: 2.145159  [57664/60000]
Test Error: 
 Accuracy: 47.6%, Avg loss: 2.133345 

Epoch 2
-------------------------------
loss: 2.140343  [   64/60000]
loss: 2.123337  [ 6464/60000]
loss: 2.061121  [12864/60000]
loss: 2.084869  [19264/60000]
loss: 2.032091  [25664/60000]
loss: 1.966189  [32064/60000]
loss: 1.987371  [38464/60000]
loss: 1.901700  [44864/60000]
loss: 1.910652  [51264/60000]
loss: 1.830600  [57664/60000]
Test Error: 
 Accuracy: 57.2%, Avg loss: 1.834249 

Epoch 3
-------------------------------
loss: 1.871719  [   64/60000]
loss: 1.831354  [ 6464/60000]
loss: 1.711824  [12864/60000]
loss: 1.758779  [19264/60000]
loss: 1.661220  [25664/60000]
loss: 1.611196  [32064/600

# Save the model

In [ ]:

# Save the model locally
torch.save(model.state_dict(), f"model-{device}.pth")
print(f"Saved PyTorch Model State to model-{device}.pth")

Saved PyTorch Model State to model-cpu.pth
